In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Direct_Stiffness_Method import BeamComponent, BoundaryCondition, BeamSolver

In [ ]:
import numpy as np

# Define inputs
nodes = np.array([
    [0.0, 0.0, 0.0, 0],      # Node 0 at (0, 0, 0)
    [15.0, 0.0, 0.0, 1],     # Node 1 at (15, 0, 0)
    [15.0, 30.0, 0.0, 2],    # Node 2 at (15, 30, 0)
    [0.0, 30.0, 0.0, 3],     # Node 3 at (0, 30, 0)
    [0.0, 0.0, 14.0, 4],     # Node 4 at (0, 0, 14)
    [15.0, 0.0, 14.0, 5],    # Node 5 at (15, 0, 14)
    [15.0, 30.0, 14.0, 6],   # Node 6 at (15, 30, 14)
    [0.0, 30.0, 14.0, 7],    # Node 7 at (0, 30, 14)
    [0.0, 0.0, 30.0, 8],     # Node 8 at (0, 0, 30)
    [15.0, 0.0, 30.0, 9],    # Node 9 at (15, 0, 30)
    [15.0, 30.0, 30.0, 10],  # Node 10 at (15, 30, 30)
    [0.0, 30.0, 30.0, 11]    # Node 11 at (0, 30, 30)
])

elements = np.array([
    [0, 4],   # E0: N0-N4
    [1, 5],   # E1: N1-N5
    [2, 6],   # E2: N2-N6
    [3, 7],   # E3: N3-N7
    [4, 8],   # E4: N4-N8
    [5, 9],   # E5: N5-N9
    [6, 10],  # E6: N6-N10
    [7, 11],  # E7: N7-N11
    [4, 5],   # E8: N4-N5
    [5, 6],   # E9: N5-N6
    [6, 7],   # E10: N6-N7
    [7, 4],   # E11: N7-N4
    [8, 9],   # E12: N8-N9
    [9, 10],  # E13: N9-N10
    [10, 11], # E14: N10-N11
    [11, 8]   # E15: N11-N8
])

# Define properties for each element (strictly with required keys)
prop_a = {
    'E': 100000,           # Young's Modulus (100000 Pa)
    'nu': 0.3,             # Poisson's Ratio
    'A': np.pi * 1.0**2,   # Cross-sectional area (approx 3.14159 m^2)
    'Iy': np.pi * 1.0**4 / 4,  # Moment of inertia about y-axis (approx 0.785398 m^4)
    'Iz': np.pi * 1.0**4 / 4,  # Moment of inertia about z-axis (approx 0.785398 m^4)
    'J': np.pi * 1.0**4 / 2    # Polar moment of inertia (approx 1.5708 m^4)
}

prop_b = {
    'E': 500000,           # Young's Modulus (500000 Pa)
    'nu': 0.3,             # Poisson's Ratio
    'A': 0.5 * 1.0,        # Cross-sectional area (0.5 m^2)
    'Iy': 1.0 * (0.5)**3 / 12,  # Moment of inertia about y-axis (approx 0.0104167 m^4)
    'Iz': 0.5 * (1.0)**3 / 12,  # Moment of inertia about z-axis (approx 0.0416667 m^4)
    'J': 0.5 * 1.0 / 12 * (0.5**2 + 1.0**2)  # Polar moment of inertia (approx 0.0520833 m^4)
}

# Assign properties to elements (E0-E7 use prop_a, E8-E15 use prop_b)
properties = [prop_a] * 8 + [prop_b] * 8

fixed_nodes = {
    0: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),  # Fixed support at node 0
    1: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),  # Fixed support at node 1
    2: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),  # Fixed support at node 2
    3: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),  # Fixed support at node 3
    4: (None, None, None, None, None, None),  # Free at node 4
    5: (None, None, None, None, None, None),  # Free at node 5
    6: (None, None, None, None, None, None),  # Free at node 6
    7: (None, None, None, None, None, None),  # Free at node 7
    8: (None, None, None, None, None, None),  # Free at node 8
    9: (None, None, None, None, None, None),  # Free at node 9
    10: (None, None, None, None, None, None), # Free at node 10
    11: (None, None, None, None, None, None)  # Free at node 11
}

loads = {
    8: (0.0, 0.0, -1.0, 0.0, 0.0, 0.0),  # Force at node 8 (downward 1 unit)
    9: (0.0, 0.0, -1.0, 0.0, 0.0, 0.0),  # Force at node 9 (downward 1 unit)
    10: (0.0, 0.0, -1.0, 0.0, 0.0, 0.0), # Force at node 10 (downward 1 unit)
    11: (0.0, 0.0, -1.0, 0.0, 0.0, 0.0)  # Force at node 11 (downward 1 unit)
}

In [ ]:
# --- Analysis and Plotting ---
# Set up the beam and boundary conditions
beam = BeamComponent(nodes, elements, properties)
bc = BoundaryCondition(fixed_nodes)
for node_id, load in loads.items():
    bc.apply_load(node_id, load)

# Run the analysis
solver = BeamSolver(beam, bc)
displacements, reactions = solver.solve()
print("Static Analysis Results:")
solver.display_results(displacements, reactions)  # Already prints all nodes as requested
print("Internal Forces per Element:")
for elem_idx, forces in solver.internal_forces.items():
    print(f"Element {elem_idx}: {np.round(forces, 5)}")

# Buckling analysis
eigvals, eigvecs, buckling_forces = solver.solve_buckling()
print("\nDisplaying First Buckling Mode:\n")

# Modified display_buckling_results to limit output
print("Buckling Analysis Results (First Mode):")
if len(eigvals) > 0:
    mode_idx = 0
    print(f"Mode 1: Critical Load Multiplier = {eigvals[mode_idx]:.5f}")
    print("Mode Shape (Displacements):")
    mode_disp = eigvecs[:, mode_idx].reshape(beam.nodes.shape[0], 6)
    for node_idx in range(beam.nodes.shape[0]):
        node_id = int(beam.nodes[node_idx, 3])
        disp = np.round(mode_disp[node_idx], 5)
        print(f"Node {node_id}: [UX: {disp[0]}, UY: {disp[1]}, UZ: {disp[2]}, RX: {disp[3]}, RY: {disp[4]}, RZ: {disp[5]}]")
    print("Internal Forces for Buckling Mode 1:")
    for elem_idx, forces in buckling_forces[mode_idx].items():
        forces = np.round(forces, 5)
        print(f"Element {elem_idx}: [Fx1: {forces[0]}, Fy1: {forces[1]}, Fz1: {forces[2]}, Mx1: {forces[3]}, My1: {forces[4]}, Mz1: {forces[5]}, "
              f"Fx2: {forces[6]}, Fy2: {forces[7]}, Fz2: {forces[8]}, Mx2: {forces[9]}, My2: {forces[10]}, Mz2: {forces[11]}]")
else:
    print("No buckling modes found.")

# Define plotting functions with silenced print statements
def plot_internal_forces(beam, internal_forces):
    # print("Plotting internal forces...")  # Silenced
    for elem_idx, forces in internal_forces.items():
        node1_id, node2_id = beam.elements[elem_idx]
        n1_idx = np.where(beam.nodes[:, 3] == node1_id)[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == node2_id)[0][0]
        n1_loc = beam.nodes[n1_idx, :3]
        n2_loc = beam.nodes[n2_idx, :3]
        length = np.linalg.norm(n2_loc - n1_loc)
        x = [0, length]
        # print(f"Element {elem_idx} forces: {forces}")  # Silenced
        
        fig, axs = plt.subplots(2, 3, figsize=(15, 8))
        titles = ['$F_x$', '$F_y$', '$F_z$', '$M_x$', '$M_y$', '$M_z$']
        indices = [(0, 6), (1, 7), (2, 8), (3, 9), (4, 10), (5, 11)]
        
        for i, (ax, title, idx_pair) in enumerate(zip(axs.flat, titles, indices)):
            y_values = [forces[idx_pair[0]], forces[idx_pair[1]]]
            ax.plot(x, y_values, 'b-o')
            ax.set_title(title)
            ax.set_xlabel('Length (m)')
            ax.set_ylabel('Force (N)' if i < 3 else 'Moment (N·m)')
            ax.grid(True)
            # print(f"{title} values: {y_values}")  # Silenced
        
        fig.suptitle(f'Internal Forces and Moments - Element {elem_idx}')
        plt.tight_layout()
        plt.savefig(f'internal_forces_elem_{elem_idx}.png')
        plt.close()

def plot_deformed_structure(beam, displacements, scale=5):
    # print("Plotting deformed structure...")  # Silenced
    print(f"Displacements: {displacements}")  # Kept as requested
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    for elem in beam.elements:
        n1_idx = np.where(beam.nodes[:, 3] == elem[0])[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == elem[1])[0][0]
        x = [beam.nodes[n1_idx, 0], beam.nodes[n2_idx, 0]]
        y = [beam.nodes[n1_idx, 1], beam.nodes[n2_idx, 1]]
        z = [beam.nodes[n1_idx, 2], beam.nodes[n2_idx, 2]]
        ax.plot(x, y, z, 'k--', label='Original' if elem[0] == 0 else "")
        # print(f"Original element {elem}: x={x}, y={y}, z={z}")  # Silenced
    
    deformed_nodes = beam.nodes[:, :3] + displacements[:, :3] * scale
    print(f"Deformed nodes: {deformed_nodes}")  # Kept as requested
    for elem in beam.elements:
        n1_idx = np.where(beam.nodes[:, 3] == elem[0])[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == elem[1])[0][0]
        x = [deformed_nodes[n1_idx, 0], deformed_nodes[n2_idx, 0]]
        y = [deformed_nodes[n1_idx, 1], deformed_nodes[n2_idx, 1]]
        z = [deformed_nodes[n1_idx, 2], deformed_nodes[n2_idx, 2]]
        ax.plot(x, y, z, 'purple', label='Deformed' if elem[0] == 0 else "")
        # print(f"Deformed element {elem}: x={x}, y={y}, z={z}")  # Silenced
    
    ax.scatter(beam.nodes[:, 0], beam.nodes[:, 1], beam.nodes[:, 2], c='blue', label='Nodes')
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title(f'3D Frame Deformed Shape (Scale Factor: {scale})')
    ax.legend()
    plt.savefig('deformed_structure.png')
    plt.close()

def hermite_shape_funcs(eta):
    """Hermite shape functions for transverse displacement (v or w) in a beam element."""
    N1 = 1 - 3*eta**2 + 2*eta**3  # Displacement at node 1
    N2 = eta - 2*eta**2 + eta**3   # Rotation at node 1
    N3 = 3*eta**2 - 2*eta**3       # Displacement at node 2
    N4 = -eta**2 + eta**3          # Rotation at node 2
    return N1, N2, N3, N4

def plot_buckling_mode(beam, eigvecs, buckling_forces, eigvals, mode_num=0, scale=5, points=50):
    # print("Plotting buckling mode...")  # Silenced
    # print(f"Input eigenvector for mode {mode_num}: {mode}")  # Silenced
    mode = eigvecs[:, mode_num]
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    for elem_idx, elem in enumerate(beam.elements):
        n1_idx = np.where(beam.nodes[:, 3] == elem[0])[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == elem[1])[0][0]
        n1_loc = beam.nodes[n1_idx, :3]
        n2_loc = beam.nodes[n2_idx, :3]
        x_ref = [n1_loc[0], n2_loc[0]]
        y_ref = [n1_loc[1], n2_loc[1]]
        z_ref = [n1_loc[2], n2_loc[2]]
        ax.plot(x_ref, y_ref, z_ref, 'k--', label='Original' if elem_idx == 0 else "")
        # print(f"Original element {elem_idx}: x={x_ref}, y={y_ref}, z={z_ref}")  # Silenced
    
    mode_full = mode.reshape(beam.nodes.shape[0], 6)
    for elem_idx, elem in enumerate(beam.elements):
        n1_idx = np.where(beam.nodes[:, 3] == elem[0])[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == elem[1])[0][0]
        n1_loc = beam.nodes[n1_idx, :3]
        n2_loc = beam.nodes[n2_idx, :3]
        L = np.linalg.norm(n2_loc - n1_loc)
        
        t = np.linspace(0, 1, points)
        x_buckled = np.linspace(n1_loc[0], n2_loc[0], points)
        y_buckled = np.linspace(n1_loc[1], n2_loc[1], points)
        z_buckled = np.linspace(n1_loc[2], n2_loc[2], points)
        
        u1, v1, w1, rx1, ry1, rz1 = mode_full[n1_idx]
        u2, v2, w2, rx2, ry2, rz2 = mode_full[n2_idx]
        
        v_vals = []
        w_vals = []
        for eta in t:
            N1, N2, N3, N4 = hermite_shape_funcs(eta)
            v = N1*v1 + N2*L*rz1 + N3*v2 + N4*L*rz2
            w = N1*w1 + N2*L*ry1 + N3*w2 + N4*L*ry2
            v_vals.append(v)
            w_vals.append(w)
        
        for i in range(points):
            y_buckled[i] += scale * v_vals[i]
            z_buckled[i] += scale * w_vals[i]
        
        ax.plot(x_buckled, y_buckled, z_buckled, 'purple', label=f'Mode {mode_num+1}' if elem_idx == 0 else "")
        # print(f"Buckled element {elem_idx}: x={x_buckled}, y={y_buckled}, z={z_buckled}")  # Silenced
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title(f'3D Frame Buckling Mode {mode_num+1} (λ = {eigvals[mode_num]:.5f}, Scale = {scale})')
    ax.legend()
    plt.savefig(f'buckling_mode_{mode_num}.png')
    plt.close()

# Generate plots
print("Starting plot generation...")
plot_internal_forces(beam, solver.internal_forces)
plot_deformed_structure(beam, displacements, scale=5)
plot_buckling_mode(beam, eigvecs, buckling_forces, eigvals, mode_num=0, scale=5, points=50)
# print("Plot generation complete. Plots saved as PNG files.")  # Silenced